# Protein Graph Analytics

In [3]:
# Install graphein if necessary:
# !pip install graphein

In [31]:
from graphein.protein.config import ProteinGraphConfig, DSSPConfig
from graphein.protein.graphs import construct_graph
from graphein.protein.edges.distance import (
    add_aromatic_interactions,
    add_disulfide_interactions,
    add_hydrophobic_interactions,
    add_peptide_bonds,
)
from graphein.protein.features.nodes import asa, rsa

config = ProteinGraphConfig(
    edge_construction_functions=[       # List of functions to call to construct edges.
        add_hydrophobic_interactions,
        add_aromatic_interactions,
        add_disulfide_interactions,
        add_peptide_bonds,
    ],
    graph_metadata_functions=[asa, rsa],  # Add ASA and RSA features.
    dssp_config=DSSPConfig(),             # Add DSSP config in order to compute ASA and RSA.
)

g = construct_graph(pdb_code="3eiy", config=config)

DEBUG:graphein.protein.graphs:Deprotonating protein. This removes H atoms from the pdb_df dataframe
DEBUG:graphein.protein.graphs:Detected 174 total nodes
INFO:graphein.protein.edges.distance:Found 413 hydrophobic interactions.
INFO:graphein.protein.edges.distance:Found: 16 aromatic-aromatic interactions
DEBUG:graphein.protein.edges.distance:1 CYS residues found. Cannot add disulfide interactions with fewer than two CYS residues.


INFO:graphein.protein.utils:Downloaded PDB file for: 3eiy


## Residue Composition

In [32]:
from graphein.protein.analysis import plot_residue_composition

fig = plot_residue_composition(g, sort_by="count", plot_type="pie")
fig.show()

In [33]:
fig = plot_residue_composition(g, sort_by="count", plot_type="bar") # TODO: stack
fig.show()

## Graph Summary Statistics
We can compute graph-theoretic summary statistics for a given protein graph

In [34]:
from graphein.protein.analysis import graph_summary

graph_summary(g)

,degree,betweenness_centrality,closeness_centrality,eigenvector_centrality,communicability_betweenness_centrality,residue_type,position,chain
node,,,,,,,,
A:SER:2,1.0,0.000000,0.178719,0.005604,0.000954,SER,2,A
A:PHE:3,6.0,0.053228,0.217337,0.036295,0.070745,PHE,3,A
A:SER:4,2.0,0.002035,0.179461,0.006685,0.003441,SER,4,A
A:ASN:5,2.0,0.000750,0.174220,0.007002,0.002090,ASN,5,A
A:VAL:6,5.0,0.017827,0.204734,0.038670,0.026333,VAL,6,A
...,...,...,...,...,...,...,...,...
A:ALA:171,2.0,0.034513,0.170108,0.008591,0.037133,ALA,171,A
A:ASN:172,2.0,0.024214,0.147863,0.001368,0.026655,ASN,172,A
A:PHE:173,3.0,0.025774,0.135156,0.000269,0.026477,PHE,173,A


In [35]:
graph_summary(g, plot=True)

## Degree Distribution

In [36]:
from graphein.protein.analysis import plot_degree_distribution

fig = plot_degree_distribution(g)
fig.show()

In [37]:
from graphein.protein.analysis import plot_degree_by_residue_type

fig = plot_degree_by_residue_type(g, normalise_by_residue_occurrence=False)
fig.show()

In [38]:
fig = plot_degree_by_residue_type(g, normalise_by_residue_occurrence=True)
fig.show()

## Edge Type Distribution

In [39]:
from graphein.protein.analysis import plot_edge_type_distribution

fig = plot_edge_type_distribution(g, plot_type="bar")
fig.show()

In [40]:
from graphein.protein.analysis import plot_edge_type_distribution

fig = plot_edge_type_distribution(g, plot_type="pie")
fig.show()

## Scatter Matrix

We can use scatter matrices to correlate node-level graph metrics (e.g. centrality measures) with node features.

First, we will plot the scatter matrix for a selection of centrality measures and the relative solvent accessibility (RSA) and absolute solvent accessibility (ASA). *N.B.* we can include these features in the plot as we have added them as metadata in the initial graph construction. Refer to cell 2 at the top of this notebook for clarity.


A summary of the function parameters:

```python

plot_graph_metric_property_correlation(
    g: nx.Graph,                                # Graph to plot
    summary_statistics: List[str] = [           # Graph theoretic metrics to include
        "degree",
        "betweenness_centrality",
        "closeness_centrality",
        "eigenvector_centrality",
        "communicability_betweenness_centrality",
    ],
    properties: List[str] = ["asa", "rsa"],     # Node features to include
    colour_by: Optional[str] = "residue_type",  # How to colour the points
    opacity: float = 0.2,                       # Opacity of markers
    diagonal_visible: bool = True,              # Whether or not to show the leading diagonal of the plot
    title: Optional[str] = None,                # Plot title
    height: int = 1000,                         # Plot height
    width: int = 1000,                          # Plot width
    font_size: int = 10,                        # Font size for axes, title and ticks
)
```

In [41]:
from graphein.protein.analysis import plot_graph_metric_property_correlation

plot_graph_metric_property_correlation(g, diagonal_visible=False)

In [42]:
plot_graph_metric_property_correlation(g, diagonal_visible=False, colour_by=None)

However, we can include more features. For instance, we can correlate these metrics with the properties defined by the ExPaSy amino acid scales. We'll plot a subset for the sake of space.

The full list of properties are:

```python
['b_factor', 'pka_cooh_alpha', 'pka_nh3', 'pka_rgroup', 'isoelectric_points', 'molecularweight', 'numbercodons', 'bulkiness', 'polarityzimmerman', 'polaritygrantham', 'refractivity', 'recognitionfactors', 'hphob_eisenberg', 'hphob_sweet', 'hphob_woods', 'hphob_doolittle', 'hphob_manavalan', 'hphob_leo', 'hphob_black', 'hphob_breese', 'hphob_fauchere', 'hphob_guy', 'hphob_janin', 'hphob_miyazawa', 'hphob_argos', 'hphob_roseman', 'hphob_tanford', 'hphob_wolfenden', 'hphob_welling', 'hphob_wilson', 'hphob_parker', 'hphob_ph3_4', 'hphob_ph7_5', 'hphob_mobility', 'hplchfba', 'hplctfa', 'transmembranetendency', 'hplc2_1', 'hplc7_4', 'buriedresidues', 'accessibleresidues', 'hphob_chothia', 'hphob_rose', 'ratioside', 'averageburied', 'averageflexibility', 'alpha_helixfasman', 'beta_sheetfasman', 'beta_turnfasman', 'alpha_helixroux', 'beta_sheetroux', 'beta_turnroux', 'coilroux', 'alpha_helixlevitt', 'beta_sheetlevitt', 'beta_turnlevitt', 'totalbeta_strand', 'antiparallelbeta_strand', 'parallelbeta_strand', 'a_a_composition', 'a_a_swiss_prot', 'relativemutability']
```

These can be accessed programmatically with:
```python
for _, d in g.nodes(data=True):
    print(d.keys())
    break
```

In [43]:
from functools import partial
from graphein.protein.features.nodes import expasy_protein_scale

# Construct the graph
config = ProteinGraphConfig(
    edge_construction_functions=[
        add_hydrophobic_interactions,
        add_aromatic_interactions,
        add_disulfide_interactions,
        add_peptide_bonds,
    ],
    node_metadata_functions=[partial(expasy_protein_scale, add_separate=True)], # Add expasy scale (add partial it so each feature is added under a separate key)
)
g = construct_graph(
    pdb_code="3eiy",
    config=config
    )

# Plot
plot_graph_metric_property_correlation(
    g,
    diagonal_visible=False,
    colour_by="residue_type",
    properties=[
        "pka_rgroup",
        "isoelectric_points",
        "bulkiness",
        "transmembranetendency",
        "coilroux",
        "relativemutability"
        ]
    )

DEBUG:graphein.protein.graphs:Deprotonating protein. This removes H atoms from the pdb_df dataframe
DEBUG:graphein.protein.graphs:Detected 174 total nodes
INFO:graphein.protein.edges.distance:Found 413 hydrophobic interactions.
INFO:graphein.protein.edges.distance:Found: 16 aromatic-aromatic interactions
DEBUG:graphein.protein.edges.distance:1 CYS residues found. Cannot add disulfide interactions with fewer than two CYS residues.


In [46]:
# Construct the graph
config = ProteinGraphConfig(
    edge_construction_functions=[
        add_hydrophobic_interactions,
    ],
    node_metadata_functions=[partial(expasy_protein_scale, add_separate=True)], # Add expasy scale (add partial it so each feature is added under a separate key)
)
g = construct_graph(
    pdb_code="3eiy",
    config=config
    )

# Plot
plot_graph_metric_property_correlation(
    g,
    diagonal_visible=False,
    colour_by="residue_type",
    summary_statistics=["degree"],
    properties=[
        "pka_rgroup",
        "isoelectric_points",
        "bulkiness",
        "transmembranetendency",
        "coilroux",
        "relativemutability"
        ]
    )

DEBUG:graphein.protein.graphs:Deprotonating protein. This removes H atoms from the pdb_df dataframe
DEBUG:graphein.protein.graphs:Detected 174 total nodes
INFO:graphein.protein.edges.distance:Found 413 hydrophobic interactions.


ValueError: Length mismatch: Expected axis has 1 elements, new values have 5 elements